<a href="https://colab.research.google.com/github/MatiTaila/hack_iia/blob/master/tensorflow_example/classify_digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network Example

Build a convolutional neural network with TensorFlow.

- Author: Aymeric Damien
- Project: https://github.com/aymericdamien/TensorFlow-Examples/

## CNN Overview

![CNN](http://personal.ie.cuhk.edu.hk/~ccloy/project_target_code/images/fig3.png)

## MNIST Dataset Overview

This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

More info: http://yann.lecun.com/exdb/mnist/

In [0]:
%tensorflow_version 2.x

from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, average_precision_score
import seaborn as sns


In [0]:
print(tf.__version__)

In [0]:
%load_ext tensorboard

In [0]:
# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).

# Training parameters.
learning_rate = 0.001
training_steps = 200
batch_size = 128
display_step = 10

# Network parameters.
conv1_filters = 32 # number of filters for 1st conv layer.
conv2_filters = 64 # number of filters for 2nd conv layer.
fc1_units = 1024 # number of neurons for 1st fully-connected layer.

In [0]:
# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [0]:
# Shuffle Training Data
indexes = np.random.permutation(x_train.shape[0])

x_train = x_train[indexes]
y_train = y_train[indexes]

# now set-aside 10% of the train_data/labels as the validation set
val_perc = 0.10
val_count = int(val_perc * x_train.shape[0])

# first pick validation set from train_data/labels
x_val = x_train[:val_count,:]
y_val = y_train[:val_count]

# leave rest in training set
x_train = x_train[val_count:,:]
y_train = y_train[val_count:]

print('Training Samples {}'.format(x_train.shape[0]))
print('Validation Samples {}'.format(x_val.shape[0]))
print('Test Samples {}'.format(x_test.shape[0]))

In [0]:
# Visualize Random Digits
rand_14 = np.random.randint(0, x_train.shape[0],14)
sample_digits = x_train[rand_14]
sample_labels = y_train[rand_14]
# code to view the images
num_rows, num_cols = 2, 7
f, ax = plt.subplots(num_rows, num_cols, figsize=(12,5),
                     gridspec_kw={'wspace':0.03, 'hspace':0.01}, 
                     squeeze=True)

for r in range(num_rows):
    for c in range(num_cols):
        image_index = r * 7 + c
        ax[r,c].axis("off")
        ax[r,c].imshow(sample_digits[image_index], cmap='gray')
        ax[r,c].set_title('No. %d' % sample_labels[image_index])
plt.show()
plt.close()

In [0]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [0]:
# Create TF Model.
class ConvNet(Model):
    # Set layers.
    def __init__(self):
        super(ConvNet, self).__init__()
        # Convolution Layer with 32 filters and a kernel size of 5.
        self.conv1 = layers.Conv2D(32, kernel_size=5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2. 
        self.maxpool1 = layers.MaxPool2D(2, strides=2)

        # Convolution Layer with 64 filters and a kernel size of 3.
        self.conv2 = layers.Conv2D(64, kernel_size=3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2. 
        self.maxpool2 = layers.MaxPool2D(2, strides=2)

        # Flatten the data to a 1-D vector for the fully connected layer.
        self.flatten = layers.Flatten()

        # Fully connected layer.
        self.fc1 = layers.Dense(1024)
        # Apply Dropout (if is_training is False, dropout is not applied).
        self.dropout = layers.Dropout(rate=0.5)

        # Output layer, class prediction.
        self.out = layers.Dense(num_classes)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = tf.reshape(x, [-1, 28, 28, 1])
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=is_training)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x



In [0]:
# Build neural network model.
conv_net = ConvNet()

In [0]:
# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [0]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        # Forward pass.
        pred = conv_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = conv_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [0]:
#Reset log folder
! rm -rf /tmp/mylogs/*

In [0]:
# Run training for the given number of steps.

writer_val = tf.summary.create_file_writer('/tmp/mylogs/validation')
writer_train = tf.summary.create_file_writer('/tmp/mylogs/train')

for step, (batch_x, batch_y) in enumerate( train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.

      start = time.time()
      run_optimization(batch_x, batch_y)
      opt_time = time.time() - start
        
      if step % display_step == 0:

        pred = conv_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)

        #Do not do this in real life use batches!
        val_pred = conv_net(x_val)
        val_loss = cross_entropy_loss(val_pred, y_val)
        val_acc = accuracy(val_pred, y_val)


        with writer_train.as_default():
        
          tf.summary.scalar("loss", loss , step=step)
          tf.summary.scalar("accuracy", acc, step=step)

        with writer_val.as_default():

        
          tf.summary.scalar("loss", val_loss , step=step)
          tf.summary.scalar("accuracy", val_acc, step=step)

        print("step: %i, loss: %f, accuracy: %f, validation loss: %f validation accuracy: %f last step time: %f ms" % (step, loss, acc, val_loss, val_acc,opt_time*1000))

In [0]:
%tensorboard --logdir /tmp/mylogs/

In [0]:
test_pred = conv_net(x_test)
print("Test Accuracy: %f" % accuracy(test_pred, y_test))

In [0]:
# Predict 5 images from validation set.
n_images = 5
test_images = x_test[:n_images]
predictions = conv_net(test_images)

# Display image and model prediction.
for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
    plt.show()
    print("Model prediction: %i" % np.argmax(predictions.numpy()[i]))


In [0]:
confusion_mtx = confusion_matrix(y_test, np.argmax(test_pred,axis=1) ) 
sns.heatmap(confusion_mtx, annot=True, fmt="d")